<a href="https://colab.research.google.com/github/mzaoualim/electricity-price-forecasting/blob/main/electricity_price_forcasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Acronymes
# electricity price forcasting: epf
# day ahead price: DAP
# day ahead market: DAM
# intraday price: IDP
# intraday market: IDM

In [ ]:
# Project Notes
  ## coding style guidlines
  '''
  > Don't Repeat Yourself
  > wrap every task within a function in a separate file for reproduction
  > Work with class/method rather than function on/in function
  > document everything {variables, calls...}
  '''
  ## Link to explore
  '''
  electricity trading and epf = profit?
  https://github.com/sandeshbhatjr/energy-prediction/blob/master/README.md
  https://energy-client-dot-energy-predictor.appspot.com/
  

  '''

In [ ]:
# Related Project
# https://github.com/dswigh/green-switch/blob/master/app.py
'''
Web-app to help decide whether to heat their home with gas or electricity
'''

In [ ]:
# Template & Updates

## Libraries

In [ ]:
# Common modules
import datetime as dt
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Motivation

The starting interest is originated from the following youtube [short](https://www.youtube.com/shorts/Sxx-dnNVOdI).

Where billionnaire James Williams caracterised the electricity market with the most Volatile (even beyond cryto) and most reliably predictable market (as it follows the laws of physics).

## Problem Statement:
Performing electricity price predicition leading to profitable trading.

## Goal
How far can a deep learning model accuratly forcast electricity price without any additionnal variable except historical price observations?

In [ ]:
# Money Speaking:
'''
How far is profitable (dollar to dollar) to use a DL model for EPF?

STRATEGY ==> ARBITRAGE
 - Performe forcasting of IDP & DAP
 - Purchase electricity in DAM and sell in IDP
 - profit = IDP - DAM - fees
'''


## Data

In [ ]:
# https://ember-climate.org/data-catalogue/european-wholesale-electricity-price-data/
'''
This dataset contains average hourly, daily and monthly 
wholesale day-ahead electricity prices for European countries. 
'''

In [ ]:
# https://pypi.org/project/elmada/
'''
elmada: Dynamic electricity carbon emission factors and prices for Europe
'''

In [ ]:
# https://pypi.org/project/python-ote/
'''
Electricity prices scraper for OTE (ote-cr.cz)
'''

In [ ]:
# https://github.com/msdogan/pyiso_scraper
'''
Using this code, one can download day ahead and realtime wholesale hourly 
and fifteen minute electricity price data from 
California Independent System Operators (CAISO) and save the price data as csv
'''

In [ ]:
# Variables
'''
> historical electricity prices
 >> Day Ahead Price
 >> Intraday Price
> dummy variables
'''

In [ ]:
# EDA of a time series ?
## Seasonality viz/detection
  ## Annually/Monthly/weekly/daily/hourly??? (justify dummy variables)
## (Partial) Autocorrelation (to justify lagged obs)
## Checking for outliers (remove or keep certain variables?)

In [ ]:
# Data (pre)processing:

## Add dummies:
  ### lagged obs
  ### day of the week
  ### weekends and national holidays
  ### hours of the day (for working/peaking/calming hours)
## normalization (0 to 1)
## interpolation of missing data
## drop duplicate obs
## training/validation/testing split

## Modeling

In [ ]:
# Model to test

# Sources
  #> https://github.com/piekarsky/Short-Term-Electricity-Price-Forecasting-at-the-Polish-Day-Ahead-Market
  #> https://github.com/dimkonto/ERC-DNN/blob/main/epfdnn.py
  #> https://github.com/vjosapreniqi/Deep-Learning-for-UK-electricity-price-prediction/blob/master/Notebooks/UK%20electricity%20forecast%20Univariate%20TS.ipynb
  #> https://github.com/abodh/Electricity-cost-forecasting-using-machine-learning-and-deep-learning-models

# > Best models to handle time series forcasting:
'''
    LSTM CNN transformer
    N-BEATS (ElementAI)
    DeepAR (Amazon)
    Spacetimeformer 
    Temporal Fusion Transformer or TFT (Google)
    ---------------------
    AleaModel
    ---------------------
    Quantile regression averaging

    {Despite its simplicity 
    it has been found to perform extremely well in practice
     - the top two performing teams in the price track of the
      Global Energy Forecasting Competition (GEFCom2014) used 
      variants of QRA -}
    ---------------------
=====> Modeling strategy? 
    Single Model
    Hybrid Model
    Stacking Ensemble Model
'''
# Seasonality treatments
'''
  daily, weekly, annually
'''
# Training horizon options?
'''
  what is best suited for day-ahead forcasting?!
'''
# Walk forward nested cross-validation!? + windows experimentations!?
# Experimentations & hyperparameter tuning?

In [ ]:
# Modeling Insights
## Training/Validation/Testing Accuracy + Viz
## Error Analysis & Viz

## Investments Simulations

In [ ]:
## Investments scenarios (with or without constraints)
  ### > Arbitrage with/without storage
## ROI analysis & Viz

## Web App Solution


In [ ]:
# Template
'''
> Variable Inputs
 - Capital
 - Financial Cost:
 => Interest Rate (loan/cost to invest)
 => Free Risk Rate (bond mostly)
 => Opportunity cost (index fund return?)
 - Investment Horizon
 => forcast the next hour, day, month?
 
> Select Model
 +> Baseline model (ARIMA) for comparaison ? ==> Only in notebook exercise!

> Results
 - Predicted ROI
 -> Of each model
 - Comparaison with competing investements.
'''
# coding style guidlines
'''
> Don't Repeat Yourself
> wrap every task within a function in a separate file for reproduction
> Work with class/methodes rather than function on/in function
> document everything {variables, calls...}
'''

In [ ]:
#Write packages_list into requirements.txt
!pip freeze > requirements.txt

## Conclusion


In [ ]:
# additionnal Variables & features selection
'''
> historical electricity prices
 >+ Categorical Variables for:
    - Weekdays
    - Holidays
    - Months/Season...
 >+ Heating degree days?
 >+ Cooling degree days?
 >+ Market Coupling (neighbouring power markets)

> investigate the effect of residual training error estimation for the day-ahead
 price forecasting task and propose an error compensation deep neural network model (ERC–DNN) 
 that focuses on the minimization of prediction error, 
 while reinforcing error stability through the integration of an autoregression module

> 
'''

## Reference
[Electricity price forcasting](https://en.wikipedia.org/wiki/Electricity_price_forecasting)

[Day-ahead electricity price prediction applying hybrid models of LSTM-based deep learning methods and feature selection algorithms under consideration of market coupling](https://arxiv.org/pdf/2101.05249.pdf)

[Error Compensation Enhanced Day-Ahead Electricity Price Forecasting](https://www.mdpi.com/1996-1073/15/4/1466)

[LSTM Implementation to predict BitCoin Closing Price](https://github.com/mzaoualim/Bitcoin_LSTM/blob/main/BTC_LSTM.ipynb)

[energy-prediction](https://github.com/sandeshbhatjr/energy-prediction/blob/master/README.md)

[energy-prediction app](https://energy-client-dot-energy-predictor.appspot.com/)